# Basic Chatbot

##### Boilerplate code

In [3]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)

### LCEL - Single Session

In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions very shortly and briefly'),
    MessagesPlaceholder("history"),
    ('human',"{input}")
])

chain = prompt_template | google_llm | StrOutputParser()

conversation = []

def chat_with_bot(payload):
    input = payload['input']
    conversation.append(HumanMessage(content=input))
    res = chain.invoke({"input": input, "history": conversation})
    conversation.append(AIMessage(content=res))
    return res

print(conversation)
res1 = chat_with_bot({"input": "What is google?"})
print(conversation)
res2 = chat_with_bot({"input": "Where is it located?"})
print(conversation)

print(res1, "\n", res2)


NameError: name 'ChatPromptTemplate' is not defined

##### LCEL - Multi session

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ChatMessageHistory

prompt_template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions very shortly and briefly'),
    MessagesPlaceholder("history"),
    ('human',"{input}")
])

chain = prompt_template | google_llm | StrOutputParser()

conversation = {}

# {sessionid: [], sessionid: []}

def chat_with_bot(payload):
    input = payload['input']
    session_id = payload['session_id']

    if session_id not in conversation:
        conversation[session_id] = ChatMessageHistory()
    
    history = conversation[session_id]

    history.add_user_message(input)
    res = chain.invoke({"input": input, "history": history.messages})
    history.add_ai_message(res)
    return res

print(conversation)
res1 = chat_with_bot({"input": "What is google?", "session_id": "user1"})
print(conversation)
res2 = chat_with_bot({"input": "Where is it located?", "session_id": "user1"})
print(conversation)

print(res1, "\n", res2)


{}
<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>
History class: <class 'list'>
{'user1': InMemoryChatMessageHistory(messages=[])}
<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>
History class: <class 'list'>
{'user1': InMemoryChatMessageHistory(messages=[])}
A search engine. 
 Please provide context. What are you asking about?


##### RunnableWithMessageHistory

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

prompt_template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions very shortly and briefly'),
    MessagesPlaceholder("history"),
    ('human',"{input}")
])

chain = prompt_template | google_llm | StrOutputParser()

store = {}

def get_session_history(session_id):
    if session_id not in store:
         store[session_id] = ChatMessageHistory()
    return store[session_id]


chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)


print(conversation)
res1 = chain_with_history.invoke(
    {"input": "What is google?"},
    config={"configurable": {"session_id": "user1"}}
)
print(conversation)
res1 = chain_with_history.invoke(
    {"input": "Where is it located?"},
    config={"configurable": {"session_id": "user1"}}
)
print(conversation)

print(res1, "\n", res2)


{'user1': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is google?', additional_kwargs={}, response_metadata={}), AIMessage(content='A search engine.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Where is it located?', additional_kwargs={}, response_metadata={}), AIMessage(content='Headquartered in Mountain View, California.', additional_kwargs={}, response_metadata={})])}
{'user1': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is google?', additional_kwargs={}, response_metadata={}), AIMessage(content='A search engine.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Where is it located?', additional_kwargs={}, response_metadata={}), AIMessage(content='Headquartered in Mountain View, California.', additional_kwargs={}, response_metadata={})])}
{'user1': InMemoryChatMessageHistory(messages=[HumanMessage(content='What is google?', additional_kwargs={}, response_metadata={}), AIMessage(content='A search engine